In [8]:
# inladen benodigde packages
import pandas as pd
import math

In [22]:
# inladen dataset met kolom voor scores
# inladen data
dataset_path = "~/share/Milena/programma-discriminatie-en-racisme/01_datapreparatie/"
dataset_empty = pd.read_csv(dataset_path + 'dataset_dummy_empty.csv', sep = ';') # voor dummy dataset importeer eerste paar instances

In [32]:
def split_annotated_data(data, n_annotators, n_annotations):
    """ functie die de data splitst. iedere zin wordt n_annotations keer toegevoegd, 
    waarbij er wordt doorgeteld aan het aantal annotators, waardoor iedere annotator (ongeveer) evenveel annnotations ontvangt.
    De data kan na het annotaten weer gemerged worden door middel van de functie merge_annotated_data.
    
    Input:
    data: een DataFrame die gesplit moet worden
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin
    
    Output:
    datasets: een lijst van n_annotators DataFrames.
    """
    # creëer lege list of lists voor de uiteindelijke output
    datasets = []
    for i in range(n_annotators):
        datasets.append([])
    
    # variabelen om bij te houden waar we zijn in het toevoegen. Een zin wordt n_annotations keer toegevoegd, 
    # beginnend bij datasets
    counter = 0
    max_counter = n_annotators -1
    
    # datapunt creëren
    for i in range(len(data)):
        datapunt = data.iloc[i]
        # het datapunt aan de nodige dataset toevoegen
        for j in range(n_annotations):
            if counter < max_counter:
                datasets[counter].append(datapunt)
                counter += 1
            else: # counter = n_annotators
                datasets[counter].append(datapunt)
                counter = 0
    
    # lijsten converteren naar df zodat ze naar .csv te converteren zijn
    for i in range(n_annotators):
        datasets[i] = pd.DataFrame(datasets[i])
    
    return datasets
    
# output genereren en kijken naar de lengte
split_data = split_annotated_data(data = dataset_empty, n_annotators = 5, n_annotations = 3)
print("Lengte te labelen per persoon na splitten:", len(split_data[4]))
print("Lengte originele dataset:", len(dataset_empty))

Lengte te labelen per persoon na splitten: 3799
Lengte originele dataset: 6333


In [33]:
# output genereren: iedere dataset naar .csv
for i, dataset in enumerate(split_data):
    dataset.to_csv('te_labellen_' + str(i) + '.csv')